In [5]:
import torch
import numpy as np

%load_ext autoreload
%autoreload 2

# extract activation map for ISIC 2017 validation set

# compute culprit score from the activations, predictions, and gt

In [ ]:
from culprit import *
import json
config = json.load(open('config_skin_alexnet.json'))
os.environ["CUDA_VISIBLE_DEVICES"]= '1'
resume = 'skinmodel/checkpoint.pth'

In [6]:
from uncertainty import *

In [49]:
experiment_saved_path = './saved'
uncty = Uncertainty(experiment_saved_path)

*** actv shape (ignore dim 0 - batch size) is: [torch.Size([64, 64, 55, 55]), torch.Size([64, 192, 27, 27]), torch.Size([64, 384, 13, 13]), torch.Size([64, 256, 13, 13]), torch.Size([64, 256, 13, 13]), torch.Size([64, 4096]), torch.Size([64, 4096]), torch.Size([64, 2])] .
*** data loaded ***
*** Flattened actv vector shape is torch.Size([150, 9346]).









  0%|          | 0/10 [00:00<?, ?it/s]Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f1d82acd5c0>>
Traceback (most recent call last):
  File "/local-scratch/weinaj/anaconda3/envs/pt10/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 717, in __del__
    self._shutdown_workers()
  File "/local-scratch/weinaj/anaconda3/envs/pt10/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 713, in _shutdown_workers
    w.join()
  File "/local-scratch/weinaj/anaconda3/envs/pt10/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f1d82aae5c0>>
Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoad

*** cumulative actv map shape: torch.Size([64, 64, 55, 55])
*** cumulative actv map shape: torch.Size([64, 192, 27, 27])
*** cumulative actv map shape: torch.Size([64, 384, 13, 13])
*** cumulative actv map shape: torch.Size([64, 256, 13, 13])
*** cumulative actv map shape: torch.Size([64, 256, 13, 13])
*** cumulative actv map shape: torch.Size([64, 4096])
*** cumulative actv map shape: torch.Size([64, 4096])
*** cumulative actv map shape: torch.Size([64, 2])









 20%|██        | 2/10 [00:17<01:11,  8.99s/it]

*** cumulative actv map shape: torch.Size([128, 64, 55, 55])
*** cumulative actv map shape: torch.Size([128, 192, 27, 27])
*** cumulative actv map shape: torch.Size([128, 384, 13, 13])
*** cumulative actv map shape: torch.Size([128, 256, 13, 13])
*** cumulative actv map shape: torch.Size([128, 256, 13, 13])
*** cumulative actv map shape: torch.Size([128, 4096])
*** cumulative actv map shape: torch.Size([128, 4096])
*** cumulative actv map shape: torch.Size([128, 2])


...








 90%|█████████ | 9/10 [01:13<00:06,  6.17s/it]

*** cumulative actv map shape: torch.Size([576, 256, 13, 13])
*** cumulative actv map shape: torch.Size([576, 256, 13, 13])
*** cumulative actv map shape: torch.Size([576, 4096])
*** cumulative actv map shape: torch.Size([576, 4096])
*** cumulative actv map shape: torch.Size([576, 2])
*** cumulative actv map shape: torch.Size([600, 64, 55, 55])
*** cumulative actv map shape: torch.Size([600, 192, 27, 27])
*** cumulative actv map shape: torch.Size([600, 384, 13, 13])









100%|██████████| 10/10 [01:14<00:00,  4.70s/it]

*** cumulative actv map shape: torch.Size([600, 256, 13, 13])
*** cumulative actv map shape: torch.Size([600, 256, 13, 13])
*** cumulative actv map shape: torch.Size([600, 4096])
*** cumulative actv map shape: torch.Size([600, 4096])
*** cumulative actv map shape: torch.Size([600, 2])


evaluation metrics {'loss': -2.9766190830866495, 'overal_acc': 0.7649999856948853, 'topk_acc': 1.0, 'class_acc': [array([0.84679089, 0.42735043])], 'class_auc': [array([0.71978907, 0.71961211])], 'confusion_matrix': [array([[409,  74],
       [ 67,  50]])]}
*** activation data saved ***
*** actv shape (ignore dim 0 - batch size) is: [torch.Size([64, 64, 55, 55]), torch.Size([64, 192, 27, 27]), torch.Size([64, 384, 13, 13]), torch.Size([64, 256, 13, 13]), torch.Size([64, 256, 13, 13]), torch.Size([64, 4096]), torch.Size([64, 4096]), torch.Size([64, 2])] .
*** data loaded ***
*** label size is torch.Size([600]), right prediction is 459.
*** feature shape is torch.Size([600, 9346]).
*** right_actv shape is torch.Size([459, 9346]), wrong_actv shape is torch.Size([141, 9346]).


In [19]:
a = None
print(a is None)

True


In [147]:
X = np.ones((3,2))
Y = np.ones((3,2))+1

In [148]:
pairwise_distances(X,Y)

array([[1.41421356, 1.41421356, 1.41421356],
       [1.41421356, 1.41421356, 1.41421356],
       [1.41421356, 1.41421356, 1.41421356]])